Traffic Sign Recognition

In [ ]:
import numpy as np                 # biblioteca NumPy pentru calcule numerice
import pandas as pd                # biblioteca Pandas pentru manipularea și analiza datelor
import os                          # modulul os pentru interacțiunea cu sistemul de operare
import cv2                         # biblioteca OpenCV pentru prelucrarea imaginilor
import warnings
import matplotlib.pyplot as plt    # biblioteca Matplotlib pentru vizualizarea datelor
import tensorflow as tf            # biblioteca TensorFlow pentru învățare adâncă
from tensorflow import keras       # modulul Keras din TensorFlow pentru construirea rețelelor neuronale
from PIL import Image              # biblioteca PIL pentru prelucrarea imaginilor
from tensorflow.keras.optimizers import Adam
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split  # train_test_split din scikit-learn pentru împărțirea datelor
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # ImageDataGenerator pentru augmentarea datelor
from tensorflow.keras.optimizers import Adam   # optimizatorul Adam din TensorFlow pentru optimizarea modelului
from sklearn.metrics import accuracy_score    # accuracy_score din scikit-learn pentru calcularea acurateței
from matplotlib import style       # style din Matplotlib pentru personalizarea stilurilor de plotare
style.use('seaborn-dark')
warnings.filterwarnings("ignore")
np.random.seed(42)


2. Prepararea datelor

In [ ]:
def read_images(directory_path):
    images = []
    labels = []

    valid_extensions = ('.jpg', '.jpeg', '.png')  # Extensiile imaginilor acceptate

    subdirectories = [subdir for subdir in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, subdir))]

    for subdir in subdirectories:
        subdirectory_path = os.path.join(directory_path, subdir)
        if not os.listdir(subdirectory_path):
            continue  # Sarim peste directoare goale

        # Iteram imaginile din subdirectoriu
        for filename in os.listdir(subdirectory_path):
            if filename.lower().endswith(valid_extensions):  # Verificam daca are extensia acceptata
                image_path = os.path.join(subdirectory_path, filename)
                image = cv2.imread(image_path)
                image = cv2.resize(image, (30, 30))  # Redimensionam imaginile la 30x30 pixeli pentru a reduce timpul de antrenare
                images.append(image)

                # Adaugam eticheta imaginii
                label = int(subdir)
                labels.append(label)

    data = np.array(list(zip(images, labels)))
    
    return data

train_data = read_images('Train')
print(f"Train Data Shape : {train_data.shape}")

In [ ]:
classes = { 
    0:'Speed limit (20km/h)',
    1:'Speed limit (30km/h)', 
    2:'Speed limit (50km/h)', 
    3:'Speed limit (60km/h)', 
    4:'Speed limit (70km/h)', 
    5:'Speed limit (80km/h)', 
    6:'End of speed limit (80km/h)', 
    7:'Speed limit (100km/h)', 
    8:'Speed limit (120km/h)', 
    9:'No passing', 
    10:'No passing veh over 3.5 tons', 
    11:'Right-of-way at intersection', 
    12:'Priority road', 
    13:'Yield', 
    14:'Stop', 
    15:'No vehicles', 
    16:'Veh > 3.5 tons prohibited', 
    17:'No entry', 
    18:'General caution', 
    19:'Dangerous curve left', 
    20:'Dangerous curve right', 
    21:'Double curve', 
    22:'Bumpy road', 
    23:'Slippery road', 
    24:'Road narrows on the right', 
    25:'Road work', 
    26:'Traffic signals', 
    27:'Pedestrians', 
    28:'Children crossing', 
    29:'Bicycles crossing', 
    30:'Beware of ice/snow',
    31:'Wild animals crossing', 
    32:'End speed + passing limits', 
    33:'Turn right ahead', 
    34:'Turn left ahead', 
    35:'Ahead only', 
    36:'Go straight or right', 
    37:'Go straight or left', 
    38:'Keep right', 
    39:'Keep left', 
    40:'Roundabout mandatory', 
    41:'End of no passing', 
    42:'End no passing veh > 3.5 tons'
}

In [ ]:
labels = train_data[:, 1].astype(int)

# Calculeaza numarul de imagini pentru fiecare eticheta
unique_labels, label_counts = np.unique(labels, return_counts=True)

plt.figure(figsize=(15,10))
plt.barh(unique_labels, label_counts)
plt.xlabel('Numar')
plt.ylabel('Label')
plt.yticks(unique_labels, [classes[label] for label in unique_labels])
plt.show()


In [ ]:
# Afiseaza 10 imagini random din setul de date
indices = np.random.choice(train_data.shape[0], size=10, replace=False)
samples = train_data[indices]

fig, axes = plt.subplots(2, 5, figsize=(12, 6))
fig.subplots_adjust(hspace=0.4)

for i, (image, label) in enumerate(samples):
    ax = axes[i // 5, i % 5]
    ax.imshow(image)
    ax.set_title(classes[label])
    ax.axis('off')

plt.show()


In [ ]:
shuffled_train = train_data.copy()
shuffled_train = np.array(shuffled_train)

In [ ]:
# Amestecam datele
train_set, test_set = train_test_split(shuffled_train, test_size=0.2, random_state=42)

# Separam input-ul (X) si output-ul/label-ul (y) din setul de antrenare
x_train = train_set[:, 0]
y_train = train_set[:, 1]

# Separam input-ul (X) si output-ul/label-ul (y) din setul de testare
x_test = test_set[:, 0]
y_test = test_set[:, 1]

# Convertim input-ul din lista de imagini in array NumPy
x_train = np.array([np.array(x) for x in x_train])
y_train = np.array([np.array(y) for y in y_train])
x_test = np.array([np.array(x) for x in x_test])
y_test = np.array([np.array(y) for y in y_test])

# Convertim output-ul din lista de etichete in array NumPy
x_train = x_train / 255.0
x_test = x_test / 255.0

print("x_train.shape:", x_train.shape)
print("x_test.shape:", x_test.shape)
print("y_train.shape:", y_train.shape)
print("y_test.shape:", y_test.shape)

# Convertim etichetele din formatul int in formatul one-hot encoding
y_train = to_categorical(y_train, num_classes=43)
y_test = to_categorical(y_test, num_classes=43)


3. Crearea modelului si evaluarea lui

In [ ]:
# Definim modelul
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu', input_shape=(30,30,3)),
    tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
    tf.keras.layers.BatchNormalization(axis=-1),
    
    tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
    tf.keras.layers.BatchNormalization(axis=-1),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(rate=0.5),
    
    tf.keras.layers.Dense(43, activation='softmax')
])


In [ ]:
lr = 0.001
epochs = 30

opt = Adam(learning_rate=lr) 
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

aug = ImageDataGenerator(
    rotation_range=15,         
    zoom_range=0.2,               
    width_shift_range=0.15,       
    height_shift_range=0.15,      
    shear_range=0.2,             
    horizontal_flip=True,         
    vertical_flip=True,          
    fill_mode="reflect"           
)


history = model.fit(aug.flow(x_train,y_train, batch_size=32), epochs=epochs, validation_data=(x_test,y_test))


In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.figure(figsize=(10, 5))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


In [ ]:
shuffled_train_dataframe = pd.DataFrame(shuffled_train)
# Salvam datele amestecate intr-un fisier CSV
shuffled_train_dataframe.to_csv('data.csv', index=False)

# Salvam modelul
model.save("saved_model.h5")

In [ ]:
def predict_image(model_path, image_path):
    # Incarcam modelul
    model = tf.keras.models.load_model(model_path)
    
    # Incarcam imaginea
    image = cv2.imread(image_path)
    image = cv2.resize(image, (30, 30))
    image = np.expand_dims(image, axis=0)
    image = image / 255.0
    
    # Facem predictia
    predictions = model.predict(image)
    predicted_label = np.argmax(predictions)
    
    return predicted_label

model_path = "saved_model.h5"
image_path = "Test/00002.png"

predicted_label = predict_image(model_path, image_path)
print("Predictie:", classes[predicted_label])
